### Importing necessary modules

In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hello/nirmala.ttf
/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/kok/kok_valid.csv
/kaggle/input/aksharantar-sampled/ak

In [3]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import gc
import random
import numpy as np 
import pandas as pd

In [4]:
if torch.cuda.is_available():
    # If CUDA is available, use a CUDA device
    device = torch.device("cuda")
else:
    # If CUDA is not available, use the CPU
    device = torch.device("cpu")

In [5]:
device

device(type='cuda')

### Preprocessing

In [6]:
traindata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv',names= ['English','Hindi'],header = None)

In [7]:
testdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_test.csv',names = ['English','Hindi'],header = None)

In [8]:
valdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_valid.csv',names = ['English','Hindi'],header = None)

In [ ]:
traindata

In [ ]:
def tokenize(word):
    tokens = []
    for x in word:
        tokens.append(x)
    return tokens

In [ ]:
max_eng_len = 0
max_hin_len = 0
test_max_eng_len = 0
test_max_hin_len = 0
val_max_eng_len = 0
val_max_hin_len = 0

In [ ]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['English']:
        temp+=1
    test_max_eng_len = max(test_max_eng_len,temp)
print(test_max_eng_len)

In [ ]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['Hindi']:
        temp +=1
    test_max_hin_len = max(test_max_hin_len,temp)
print(test_max_hin_len)

In [ ]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['English']:
        temp+=1
    val_max_eng_len = max(val_max_eng_len,temp)
print(val_max_eng_len)

In [ ]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['Hindi']:
        temp+=1
    val_max_hin_len = max(val_max_hin_len,temp)
print(val_max_hin_len)

In [ ]:
English_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['English']:
        temp += 1
        if y not in English_vocab:
            English_vocab.append(y)
    if(temp>max_eng_len):
        max_eng_len = max(max_eng_len,temp)
print(sorted(English_vocab))
print(max_eng_len)

In [ ]:
Hindi_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['Hindi']:
        temp += 1
        if y not in Hindi_vocab:
            Hindi_vocab.append(y)
    max_hin_len = max(temp,max_hin_len)
for x in range(len(testdata)):
    for y in testdata.iloc[x]['Hindi']:
        if y not in Hindi_vocab:
            Hindi_vocab.append(y)


In [ ]:
len(Hindi_vocab)

In [ ]:
English_vocab = sorted(English_vocab)
Hindi_vocab = sorted(Hindi_vocab)

In [ ]:
Eng_dict = {}
reverse_Eng = {}

for x in range(len(English_vocab)):
    Eng_dict[English_vocab[x]] = x+3
    reverse_Eng[x+3] = English_vocab[x]
Eng_dict['<sow>'] = 0
Eng_dict['<eow>'] = 1
Eng_dict['<pad>'] = 2
reverse_Eng[0] = '<sow>'
reverse_Eng[1] = '<eow>'
reverse_Eng[2] = '<pad>'

print(Eng_dict)
print(reverse_Eng)

In [ ]:
Hin_dict = {}
reverse_Hin = {}
for x in range(len(Hindi_vocab)):
    Hin_dict[Hindi_vocab[x]] = x+3
    reverse_Hin[x+3] = Hindi_vocab[x]
Hin_dict['<sow>'] = 0
Hin_dict['<eow>'] = 1
Hin_dict['<pad>'] = 2
reverse_Hin[0] = '<sow>'
reverse_Hin[1] = '<eow>'
reverse_Hin[2] = '<pad>'
print(Hin_dict)
print(reverse_Hin)

In [ ]:
def Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens

In [ ]:
def Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),max_hin_len+1):
        tokens.append(Hin_dict['<pad>'])
    return tokens

In [ ]:
eng_word = []
hin_word = []
for x in range(len(traindata)):
    eng_word.append(Eng_tokenize(traindata.iloc[x]['English']))
    hin_word.append(Hin_tokenize(traindata.iloc[x]['Hindi']))

In [ ]:
eng_word = torch.tensor(eng_word)
hin_word = torch.tensor(hin_word)

In [ ]:
max_hin_len += 1
test_max_hin_len += 1
val_max_hin_len += 1

In [ ]:
def test_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),test_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def test_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),test_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
def val_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),val_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def val_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x]) 
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),val_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
val_eng_word = []
val_hin_word = []
for x in range(len(valdata)):
    val_eng_word.append(val_Eng_tokenize(valdata.iloc[x]['English']))
    val_hin_word.append(val_Hin_tokenize(valdata.iloc[x]['Hindi']))
val_eng_word = torch.tensor(val_eng_word)
val_hin_word = torch.tensor(val_hin_word)
test_eng_word = []
test_hin_word = []
for x in range(len(testdata)):
    test_eng_word.append(test_Eng_tokenize(testdata.iloc[x]['English']))
    test_hin_word.append(test_Hin_tokenize(testdata.iloc[x]['Hindi']))
test_eng_word = torch.tensor(test_eng_word)
test_hin_word = torch.tensor(test_hin_word)

### Encoder and Decoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,rnn):
        super(Encoder,self).__init__()
        self.layer = no_of_layers
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Eng_dict),char_embed_size).to(device)
        self.embedding.weight.requires_grad = True
        self.drop = nn.Dropout(dropout)
        self.LSTM = nn.LSTM(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.RNN = nn.RNN(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.GRU = nn.GRU(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
    def forward(self,input,hidden,cell):
        embedded = self.embedding(input)
        embedded1 = self.drop(embedded)
        cell1 = cell
        if(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded1,hidden)
        elif(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded1,(hidden,cell))
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded1,hidden)
        return output,(hidden1,cell1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn):
        super(Decoder,self).__init__()
        self.layer = no_of_layers
        self.batchsize = batchsize
        self.hidden_size = hidden_size
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Hin_dict),char_embed_size).to(device)
        self.drop = nn.Dropout(dropout)
        self.embedding.weight.requires_grad = True
        self.LSTM = nn.LSTM(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.RNN = nn.RNN(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.GRU = nn.GRU(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        #2*hidden_size
        self.linear = nn.Linear(hidden_size,len(Hin_dict),bias=True).to(device)
        # dim = 2 
        self.softmax = nn.Softmax(dim = 2).to(device)
    def forward(self,input,hidden,cell,OGhidden,matrix):
        embedded = self.embedding(input)
        s1 = OGhidden.size()[1]
        s2 = OGhidden.size()[2]
        embedded1 = torch.cat((embedded,OGhidden[0].resize(s1,1,s2),OGhidden[1].resize(s1,1,s2)),dim = 2)
        embedded2 = self.drop(embedded1)
        cell1 = cell
        if(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded2,(hidden,cell))
        elif(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded2,hidden)
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded2,hidden)
        output1 = self.linear(output)
        return output1,(hidden1,cell1)
        output2 = self.softmax(output1)
        return output2,hidden11
        
    #changed GRU char_embed_size
    #changed forward embedded

In [ ]:
def getword(characters):
    return "".join(characters)

In [ ]:
def accuracy(target,predictions,flag):
    total = 0
    for x in range(len(target)):
        if(torch.equal(target[x],predictions[x])):
            total += 1
    return total

In [ ]:
def translate(target,predictions,df):
    i = len(df)
    for x in range(len(predictions)):
        original = []
        for y in target[x]:
            if(y != 1):
                original.append(y)
            else:
                break
        predicted = []
        for y in predictions[x]:
            if(y != 1):
                predicted.append(y)
            else:
                break
        df.loc[i,['Original']] = getword([reverse_Hin[x.item()] for x in original])
        df.loc[i,['Predicted']] = getword([reverse_Hin[x.item()] for x in predicted])
        i+=1
    return df
    

In [ ]:
def Evaluate(attention,test_eng_word,test_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        df = pd.DataFrame()
        en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        for x in range(0,len(testdata),batchsize):
            loss = 0
            input_tensor = test_eng_word[x:x+batchsize].to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(input_tensor)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
            OGhidden = hidden1
            predicted = []
            predictions = []
            if(attention == True):
                temp = output
            else:
                temp = OGhidden
            for i in range(test_max_hin_len):
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,temp,False)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(test_max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(test_hin_word[x:x+batchsize].to(device),predictions,x)
            df = translate(test_hin_word[x:x+batchsize],predictions,df)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,test_hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
        test_loss = total_loss/(len(testdata)*test_max_hin_len)
        test_accuracy = (total_acc/len(testdata))*100
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        return test_loss,test_accuracy,df

In [ ]:
def valevaluate(attention,val_eng_word,val_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        for x in range(0,len(valdata),batchsize):
            loss = 0
            input_tensor = val_eng_word[x:x+batchsize].to(device)
#             en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(input_tensor)
            del(en_hidden)
            del(en_cell)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
#             hidden1 = hidden[0]
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
#             cell1 = cell[0]
            OGhidden = hidden1
            predicted = []
            predictions = []
            if(attention == True):
                temp = output
            else:
                temp = OGhidden
            for i in range(val_max_hin_len):
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,temp,False)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(val_max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(val_hin_word[x:x+batchsize].to(device),predictions,x)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,val_hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
#             print(loss.item())
        validation_loss = total_loss/(len(valdata)*val_max_hin_len)
        validation_accuracy = (total_acc/len(valdata))*100
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        return validation_loss,validation_accuracy

In [ ]:
def train(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn):
    gc.collect()
    torch.autograd.set_detect_anomaly(True)
    encoder = Encoder(char_embed_size,hidden_size,no_of_layers,dropout,rnn).to(device)
    decoder = Decoder(char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn).to(device)
    print(encoder.parameters)
    print(decoder.parameters)
    opt_encoder = optim.Adam(encoder.parameters(),lr = 0.001)
    opt_decoder  = optim.Adam(decoder.parameters(),lr = 0.001)
    teacher_ratio = 0.5
#     en_hidden = torch.randn(2*no_of_layers,batchsize,hidden_size).to(device)
    for _ in range(epochs):
        print(_)
        total_loss = 0
        total_acc = 0
        for x in range(0,len(traindata),batchsize):
            loss = 0
            opt_encoder.zero_grad()
            opt_decoder.zero_grad()
            input_tensor = eng_word[x:x+batchsize].to(device)
            en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(en_hidden)
            del(en_cell)
            del(input_tensor)
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
            cell1 = torch.add(cell[0],cell[1])/2
            OGhidden = hidden1
            predicted = []
            predictions = []
            use_teacher_forcing = True if random.random() < teacher_ratio else False
            if use_teacher_forcing:
                for i in range(max_hin_len):
                    output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,OGhidden,False)
                    predicted.append(output1)
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    predictions.append(output3)
                    input2 = hin_word[x:x+batchsize,i].to(device).resize(batchsize,1)
            else:
                for i in range(max_hin_len):
                    output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,OGhidden,False)
                    predicted.append(output1)
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    predictions.append(output3)
                    input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(hin_word[x:x+batchsize].to(device),predictions,x)
#             print(predicted.shape)
#             print(hin_word[x:x+batchsize].reshape(-1).shape)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
            loss.backward(retain_graph = True)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            opt_encoder.step()
            opt_decoder.step()
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        training_loss = total_loss/(51200*max_hin_len)
        training_accuracy = total_acc/512
        validation_loss,validation_accuracy = valevaluate(False,val_eng_word,val_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers)
#         wandb.log({'training_accuracy' : training_accuracy, 'validation_accuracy' : validation_accuracy,'training_loss' : training_loss, 'validation_loss' : validation_loss,'epoch':_+1})
#         if(_ >= epochs/2):
#             teacher_ratio = 0
    return encoder,decoder

In [34]:
# import wandb
# wandb.login()

In [35]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=be82296761cb21e90ce245472090fa945a6756c6e77e4ed1209ef42713b22e7f
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Best configuration


In [36]:
batchsize = 128
hidden_size = 1024
char_embed_size = 128
no_of_layers = 2
dropout = 0.5
epochs = 20
rnn = 'LSTM'

In [37]:
Encoder1,Decoder1 = train(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn)

<bound method Module.parameters of Encoder(
  (embedding): Embedding(29, 128)
  (drop): Dropout(p=0.5, inplace=False)
  (LSTM): LSTM(128, 1024, num_layers=2, batch_first=True, bidirectional=True)
  (RNN): RNN(128, 1024, num_layers=2, batch_first=True, bidirectional=True)
  (GRU): GRU(128, 1024, num_layers=2, batch_first=True, bidirectional=True)
)>
<bound method Module.parameters of Decoder(
  (embedding): Embedding(68, 128)
  (drop): Dropout(p=0.5, inplace=False)
  (LSTM): LSTM(2176, 1024, num_layers=2, batch_first=True)
  (RNN): RNN(2176, 1024, num_layers=2, batch_first=True)
  (GRU): GRU(2176, 1024, num_layers=2, batch_first=True)
  (linear): Linear(in_features=1024, out_features=68, bias=True)
  (softmax): Softmax(dim=2)
)>
0


/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [38]:
test_loss,test_accuracy,predictions = Evaluate(False,test_eng_word,test_hin_word,Encoder1,Decoder1,batchsize,hidden_size,char_embed_size,no_of_layers)

/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.10/site-packages/

In [41]:
test_accuracy

37.744140625

In [39]:
#combine predictions and english column of testdata and show just head(10)

In [42]:
predictions.to_excel("output.xlsx")

In [19]:
input = pd.DataFrame(testdata.iloc[:]['English'])

In [10]:
input['index'] = input.index

In [11]:
input

,English,index
0,thermax,0
1,sikhaaega,1
2,learn,2
3,twitters,3
4,tirunelveli,4
...,...,...
4091,saflata,4091
4092,shbana,4092
4093,khaatootolaa,4093
4094,shivastava,4094


In [20]:
input = pd.DataFrame(testdata.iloc[:]['English'])
input['index'] = input.index
df2['index'] = df2.index
result = pd.merge(input,df2)

In [21]:
result

,English,index,Unnamed: 0,Original,Predicted
0,thermax,0,0,थरमैक्स,थर्मक्स
1,sikhaaega,1,1,सिखाएगा,सिखाएगा
2,learn,2,2,लर्न,लियरन
3,twitters,3,3,ट्विटर्स,ट्विटर्स
4,tirunelveli,4,4,तिरुनेलवेली,तिरुनेलवेली
...,...,...,...,...,...
4091,saflata,4091,4091,सफ़लता,सफलता
4092,shbana,4092,4092,शबाना,श्बाना
4093,khaatootolaa,4093,4093,खातूटोला,खातूतोला
4094,shivastava,4094,4094,शिवास्तव,शिवास्तवा


In [18]:
result.drop()

,English,index,Unnamed: 0,Original,Predicted
0,thermax,0,0,थरमैक्स,थर्मक्स
1,sikhaaega,1,1,सिखाएगा,सिखाएगा
2,learn,2,2,लर्न,लियरन
3,twitters,3,3,ट्विटर्स,ट्विटर्स
4,tirunelveli,4,4,तिरुनेलवेली,तिरुनेलवेली
...,...,...,...,...,...
4091,saflata,4091,4091,सफ़लता,सफलता
4092,shbana,4092,4092,शबाना,श्बाना
4093,khaatootolaa,4093,4093,खातूटोला,खातूतोला
4094,shivastava,4094,4094,शिवास्तव,शिवास्तवा


In [63]:
predictions

,Original,Predicted,index
0,थरमैक्स,थेरमैक्स,0
1,सिखाएगा,सिखाएगा,1
2,लर्न,लियरन,2
3,ट्विटर्स,ट्विटर्स,3
4,तिरुनेलवेली,तिरूनलेलवी,4
...,...,...,...
4091,सफ़लता,सफलता,4091
4092,शबाना,श्बाना,4092
4093,खातूटोला,खातूूतला,4093
4094,शिवास्तव,शिवास्तवा,4094


In [65]:
result = pd.merge(input,predictions)

In [67]:
result = result.drop(['index'],axis = 1)

In [2]:
result2 = result.head(20)

NameError: name 'result' is not defined

In [44]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [73]:
wandb.init(project='CS6910_DLAssignment3')
wandb.log({'result' : result})

## Confusion Matrix

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/kaggle/input/predictions/wandb_export_2023-05-20T22_42_50.44605_30.csv")

In [6]:
df

,English,Original,Predicted
0,thermax,थरमैक्स,थेरमैक्स
1,sikhaaega,सिखाएगा,सिखाएगा
2,learn,लर्न,लियरन
3,twitters,ट्विटर्स,ट्विटर्स
4,tirunelveli,तिरुनेलवेली,तिरूनलेलवी
...,...,...,...
4091,saflata,सफ़लता,सफलता
4092,shbana,शबाना,श्बाना
4093,khaatootolaa,खातूटोला,खातूूतला
4094,shivastava,शिवास्तव,शिवास्तवा


In [92]:
target = []
predicted  = []
for x in range(len(df.iloc[:]['Original'])):
    if(len(df.iloc[x]['Original']) < len(df.iloc[x]['Predicted'])):
        for y in range(len(df.iloc[x]['Original'])):
            target.append(Hin_dict[df.iloc[x]['Original'][y]]-3)
            predicted.append(Hin_dict[df.iloc[x]['Predicted'][y]]-3)
    else:
        for y in range(len(df.iloc[x]['Predicted'])):
            target.append(Hin_dict[df.iloc[x]['Original'][y]]-3)
            predicted.append(Hin_dict[df.iloc[x]['Predicted'][y]]-3)
        
        

In [96]:
max(target)

64

In [94]:
max(predicted)

64

In [72]:
len(Hindi_vocab)

65

In [102]:
wandb.init(project='CS6910_DLAssignment3')
wandb.log({"conf_mat" : wandb.sklearn.plot_confusion_matrix(target,predicted,Hindi_vocab)})

wandb: WARNING using only the first 1000 datapoints to create chart confusion_matrix
